In [33]:
import os
runner_path = "/home/alexmak123/neurone"
config_path = "/home/alexmak123/neurone/configs/endometritis.json"
weigths_path = "/home/alexmak123/neurone/experiments/endonet_v1/epoch_93.pth"
engine_type = "gpu"
device_number = 3
%cd $runner_path

if (engine_type == "gpu"):
    #here we choose on which tesla to run
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "3"
    !export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

import argparse
import json
from animus.torch.engine import CPUEngine, DDPEngine, DPEngine, GPUEngine, XLAEngine
from neurone.utils.general import load_yaml
import torch
from datetime import datetime 
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from PIL import Image, ImageDraw
from skimage.io import imread, imsave, imshow
import cv2
%matplotlib inline

/home/alexmak123/neurone


In [34]:
#Download our tiles
tiles_directory_in_str = "/home/alexmak123/result_tiles_for_validation/21-856-1CD138"
%cd $tiles_directory_in_str

alb_transforms = A.Compose([A.augmentations.Resize(1024, 1024)])
def import_tiles_from_directory_and_convert_them (tiles_directory_in_str):
    directory = os.fsencode(tiles_directory_in_str)
    converted_images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if (filename.endswith(".png") or filename.endswith(".jpg")): 
            myTile = np.array(Image.open(filename).convert('RGB'))
            myTile = alb_transforms(image=myTile)["image"]
            converted_images.append(myTile)
            continue
        else:
            continue
    return np.array(converted_images)

converted_images = import_tiles_from_directory_and_convert_them (tiles_directory_in_str)
print(converted_images.shape)

/home/alexmak123/result_tiles_for_validation/21-856-1CD138
(3969, 1024, 1024, 3)


In [35]:
# Runners
from neurone.runners.detection import BaseDetectionRunner, HeatmapDetectionRunner
from neurone.runners.segmentation import BaseSegmentationRunner
from neurone.runners.gridsearch import GridSearchRunner


E2E = {
    "cpu": CPUEngine,
    "gpu": GPUEngine,
    "dp": DPEngine,
    "ddp": DDPEngine,
    "xla": XLAEngine,
}


with open(config_path, "r") as config_file:
        config = json.load(config_file)
config["train"]["engine"] = engine_type

if config["train"]["deterministic"]:
    torch.use_deterministic_algorithms(config["train"]["deterministic"])
logging_dir = os.path.join(config["train"]["save_dir"], config["train"]["experiment_name"])
    
runner = globals()[config["model"]["runner"]](engine=E2E[engine_type](
    log_with="tensorboard", logging_dir=logging_dir), config=config)

runner._setup_model()
runner.model.eval()


#if we have gpu, we should run on cuda
if (engine_type == "gpu"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    runner.model.to(device)

In [36]:
import sys

#our data to tensor
image_torch = torch.tensor(converted_images)
#image_torch = torch.unsqueeze(image_torch, 0)
image_torch = image_torch.permute(0, 3, 1, 2) 
image_torch = image_torch.float()
image_torch -= torch.tensor([config["data"]["norm_mean"][0], config["data"]["norm_mean"][1], config["data"]["norm_mean"][2]]).reshape(-1, 1, 1)
image_torch /= torch.tensor([config["data"]["norm_std"][0], config["data"]["norm_std"][1], config["data"]["norm_std"][2]]).reshape(-1, 1, 1)


if (engine_type == "cpu"):
    all_keypoints = runner.model(image_torch)["keypoints"]
    #confidences = runner.model(image_torch)["confidences"]

if (engine_type == "gpu"):
    torch.cuda.empty_cache()
    all_keypoints = []
    #all_confidences = []
    for img in image_torch:
        img = img.to(device)
        img = torch.unsqueeze(img, 0)
        keypoint = np.array(runner.model(img)["keypoints"])
        all_keypoints.append(keypoint)
        #confidence = runner.model(pic)["confidences"]
        #all_confidences.append(confidence)
        

In [29]:
#split keypoints of all tiles to array of selected colums with x_coords, y_coords and classes of all tiles
if (engine_type == "cpu"):
    j = 0
    selected_columns_array = []
    for index_tile in range(converted_images.shape[0]):
        selected_columns = []
        while (j < all_keypoints.shape[0] and all_keypoints[j][0] == index_tile):
            selected_columns.append([float(all_keypoints[j][1]), float(all_keypoints[j][2]), float(all_keypoints[j][3])])
            j += 1
        selected_columns_array.append(np.array(selected_columns))

if (engine_type == "gpu"):
    selected_columns_array = []
    for keypoint in all_keypoints:
        selected_columns_array.append(np.array(keypoint[:, [1, 2, 3]]))

In [37]:
#ConvexHull
#Is used to find clusters of dots on the entire slide
#We add median filter to reduce some noise 
#We use laplacian filter for better precision 


plt.rcParams["figure.figsize"] = [20, 20]
plt.rcParams["figure.autolayout"] = True


#def that gets rid of hulls that are noise 
def reduce_noise(hulls, noise_threshold = 35):
    hulls_without_noise = []
    for curr_hull in hulls:
        if (cv2.arcLength(curr_hull, True) >= noise_threshold):
            hulls_without_noise.append(curr_hull)
            
    return hulls_without_noise


#def that removes duplicates of the contours (it is happend because cv2.findContours returns clockwise and non-clockwise oriented contours) 
def get_only_positive_oriented_contours(contours):
    positive_oriented_contours = []
    for curr_contour in contours:
        if (cv2.contourArea(curr_contour, 1) <= 0 and cv2.contourArea(curr_contour, 0) >= 0):
            positive_oriented_contours.append(curr_contour)
    
    return positive_oriented_contours


#def that groups convex hulls in clusters of nested convex hulls (presented as a graph)
def group_convex_hulls(hulls):
    hierarchy = []
    for i in range(len(hulls)):
        #-1 stands for absence of the parent, this means that this convex hull is the outermost of the cell
        parent = -1
        for j in range(len(hulls)):
            if i == j:
                continue
            if cv2.pointPolygonTest(hulls[j], tuple(hulls[i][0][0]), False) >= 0:
                parent = j
                break
        hierarchy.append([i, parent])
    return hierarchy


#def that finds closest to outer hull, inner hull (if exists)
def find_closest_inner_hull(hierarchy, outer_hull_index):
    inner_hull_index = -2
    for (index, parent) in hierarchy : 
        if (parent == outer_hull_index):
            inner_hull_index = index
            break
            
    return inner_hull_index   


#def that counts mean hsv color of points, bounded by inner(if exists) and outer convex hull 
def coordinates_of_target_pixels_between_two_hulls(curr_picture, hulls, hierarchy, outer_hull_index):
    cv2.fillConvexPoly(curr_picture, hulls[outer_hull_index], (255, 0, 0))
    
    #lets find closest to outer hull, inner hull index (if it's -2, then we don't have an inner_hull)
    inner_hull_index = find_closest_inner_hull(hierarchy, outer_hull_index)
    if (inner_hull_index != -2): 
        cv2.fillConvexPoly(curr_picture, hulls[inner_hull_index], (0, 255, 0))
    
    target_value = (255, 0, 0)
    coordinates_of_target_pixels = []
    #quite seriously increases the running time of our program, it is better not to use it if it is not necessary
    for y in range(curr_picture.shape[0]):
        for x in range(curr_picture.shape[1]):
            if np.all(curr_picture[y, x] == target_value):
                coordinates_of_target_pixels.append((x, y))
                
    # Displaying the image
    imshow(curr_picture.astype('uint8'), vmin=0, vmax=255)  
    return coordinates_of_target_pixels


#def that counts hsv color of target_pixels in the original image
import colorsys
def count_hsv(original_image, coordinates_of_target_pixels):
    pixels = []
    for (y,x) in coordinates_of_target_pixels:
        pixels.append(original_image[x][y])
    
    rgb_mean = np.mean(pixels, axis = 0)
    (rgb_mean[0], rgb_mean[1], rgb_mean[2]) = (rgb_mean[0] / float(255), rgb_mean[1] / float(255), rgb_mean[2] / float(255))
    hsv = colorsys.rgb_to_hsv(rgb_mean[0], rgb_mean[1], rgb_mean[2])
    hsv = (int(hsv[0] * 179), int(hsv[1] * 255), int(hsv[2] * 255))
    
    return hsv


#def that counts the amount of stroma cells inside the convex hull. If there is one cell, then we will say that this cell is plasmatic
def count_the_amount_of_cells_inside_the_convex_hull (keypoints_for_this_img, hulls, outer_hull_index):
    amount_of_cells = 0
    outer_hull = hulls[outer_hull_index]
    center_keypoints = []
    for keypoint in keypoints_for_this_img:
        if (keypoint[2] == 0):
            x = keypoint[0]
            y = keypoint[1]
            distances = cv2.pointPolygonTest(outer_hull, (x,y), measureDist=False)
            if (distances >= 0):
                amount_of_cells += 1
                center_keypoints.append([x,y])
            
    return center_keypoints, amount_of_cells 


#def that is our main convex hull function 
def convex_hull_for_showing(images_list, path_to_save, name_of_the_slide, median_filter_size = 5):
    hsv_brown_cells = []
    for i in range(len(images_list)):
        
        keypoints_for_this_img = selected_columns_array[i]
        
        median_image  = cv2.medianBlur(images_list[i], median_filter_size)
        
        img_grey = cv2.cvtColor(median_image, cv2.COLOR_BGR2GRAY)
        (thresh, blackAndWhiteImage) = cv2.threshold(img_grey, 115, 255, cv2.THRESH_BINARY)
        kernel = np.array([[0,1,0], [1,-4,1], [0,1,0]])
        img = cv2.filter2D(blackAndWhiteImage, -1, kernel)
        
        contours,_  = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
        
        #get rid of duplication
        contours = get_only_positive_oriented_contours(contours)
        
        #count hulls
        hulls = []
        img_hulls_and_contours = np.zeros(images_list[i].shape)
        for j in range(len(contours)):
            cv2.drawContours(img_hulls_and_contours, contours, j, (0, 255, 0), 1, 8)
            hulls.append(cv2.convexHull(contours[j], False))
            
        #reduce noise in the hulls
        hulls = reduce_noise(hulls)
        
        #group convex hulls in nested convex hulls
        hierarchy = group_convex_hulls(hulls)
        
        #print everything 
        print ("curr_pic" + str(i))
        for (index, parent) in (hierarchy):        
            if (parent == -1):
                center_keypoints, amount = count_the_amount_of_cells_inside_the_convex_hull(keypoints_for_this_img, hulls, index)
                print (amount)
                if (amount == 1):
                    #draw a circle on the plasmatic cell and save this image
                    PIL_image = Image.fromarray(images_list[i].astype('uint8'), 'RGB')
                    PIL_image.save(os.path.join(path_to_save,
                            "save_original_tiles", name_of_the_slide, name_of_the_slide + "_" + str(i) + '.png'), "PNG")
                    cv2.circle(images_list[i], (int(center_keypoints[0][0]), int(center_keypoints[0][1])), 4, (255,0,0), -1)
                    PIL_image = Image.fromarray(images_list[i].astype('uint8'), 'RGB')
                    PIL_image.save(os.path.join(path_to_save,
                            name_of_the_slide, name_of_the_slide + "_" + str(i) + '.png'), "PNG")
                

In [38]:
#def that makes the given folder empty
def empty_folder(folder):
    """Makes given folder empty"""
    import os, shutil
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

            
#def that makes needed directories         
def make_directories (tiles_layout_path, path_to_save, name_of_the_slide, preserve = False):
    if not os.path.isdir(tiles_layout_path):
        raise BaseException("No such directory: " + tiles_layout_path)
            
    if not os.path.isdir(path_to_save):
        print("Creating directory " + path_to_save + "...")
        os.mkdir(path_to_save)
        
    if not os.path.isdir(os.path.join(path_to_save, name_of_the_slide)):
        print("Creating directory " + name_of_the_slide + "...")
        os.mkdir(os.path.join(path_to_save, name_of_the_slide))
        os.mkdir(os.path.join(path_to_save, "save_original_tiles", name_of_the_slide))
    """    
    else:
        print('Destination folder is not empty')
        if not preserve:
            print('Removing all files...')
            empty_folder(path_to_save)
    """

In [39]:
tiles_layout_path = "/home/alexmak123/result_tiles_for_validation/21-856-1CD138"
#NOTE: in PATH_TO_SAVE directory all the data will be deleted
PATH_TO_SAVE = "/home/alexmak123/test_cappa_koef/save_plasmatic"

slides = os.listdir(tiles_layout_path)
slide_to_check = "21-856-1CD138"      

make_directories(tiles_layout_path, PATH_TO_SAVE, slide_to_check) 

convex_hull_for_showing(converted_images, PATH_TO_SAVE, slide_to_check)

Creating directory 21-856-1CD138...
curr_pic0
curr_pic1
curr_pic2
curr_pic3
curr_pic4
curr_pic5
0
curr_pic6
curr_pic7
curr_pic8
curr_pic9
curr_pic10
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
curr_pic11
curr_pic12
curr_pic13
curr_pic14
curr_pic15
curr_pic16
curr_pic17
curr_pic18
curr_pic19
curr_pic20
0
curr_pic21
curr_pic22
curr_pic23
curr_pic24
curr_pic25
curr_pic26
curr_pic27
curr_pic28
curr_pic29
curr_pic30
curr_pic31
curr_pic32
curr_pic33
curr_pic34
curr_pic35
curr_pic36
curr_pic37
curr_pic38
0
curr_pic39
curr_pic40
curr_pic41
curr_pic42
curr_pic43
0
0
0
0
0
0
0
0
curr_pic44
0
curr_pic45
curr_pic46
curr_pic47
curr_pic48
curr_pic49
curr_pic50
curr_pic51
curr_pic52
curr_pic53
curr_pic54
curr_pic55
curr_pic56
curr_pic57
curr_pic58
curr_pic59
curr_pic60
curr_pic61
curr_pic62
curr_pic63
curr_pic64
curr_pic65
0
0
0
0
0
0
0
curr_pic66
curr_pic67
curr_pic68
0
0
0
curr_pic69
curr_pic70
curr_pic71
0
0
curr_pic72
curr_pic73
curr_pic74
curr_pic75
curr_pic76
curr_pic77
curr_pic78
curr_pic79
curr_pic80